In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import shutil
from sklearn.model_selection import train_test_split
import glob
import random
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

import time
import datetime
import sys

from torch.autograd import Variable
import torch
import numpy as np
import itertools
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import matplotlib.animation  as animation
from IPython.display import HTML , Image as ImgDisplay # burada sorun çıkabilir çıkarsa from IPython.display import Image as ImgDisplay olacak şekilde aşağıya yaz 
from torchsummary import summary
from torchvision.utils import save_image
import pandas as pd

In [ ]:
image_dir = "./img_align_celeba/img_align_celeba"
attributes_file = "./list_attr_celeba.csv"
output_dir = "./preprocessed_dataset_celeba"

data = pd.read_csv(attributes_file)

In [ ]:
# Siyah saç özelliği "1" olan Kadın ünlülerden 1000 tane örnek seçtik, dataframe'e attık.
df_blackHair = data.loc[data["Black_Hair"] == 1 & (data["Male"] == -1)].sample(n = 1000) 
df_blonde = data.loc[data["Blond_Hair"] == 1 & (data["Male"] == -1)].sample(n = 1000)

#Sarı saç özelliği "1" olan Kadın ünlülerden 1000 tane örnek seçtik, dataframe'e attık.

In [ ]:
domainK, domainD = [],[]

for index, row in df_blackHair.iterrows():
    domainK.append(row["image_id]")

for index, row in df_blonde.iterrows():
    domainK.append(row["image_id]")



In [ ]:
# Train ve test verilerini ilk seçtiğimiz 1000'er resimden bölerek oluşturalım.
K_train, K_test = train_test_split(domainK, test_size = 0.01, random_state = 1453 )
D_train, D_test = train_test_split(domainD, test_size = 0.01, random_state = 1453 )

Eğitim ve test veri seti başka bir yere kaydediliyor

In [ ]:
N = min(len(K_train), len(B_train))
K_train = K_train[:N]
D_train = D_train[:N]
print(f"Images in K(black) {len(K_train)} and D(blonde) {len(D_train)}")
K_train_dir = os.path.join(output_dir, "train/K")
D_train_dir = os.path.join(output_dir, "train/D")

os.makedirs(K_train_dir, exist_ok = True)
os.makedirs(D_train_dir, exist_ok = True)

for imageK, imageD in zip(K_train, D_train):
    shutil.copy(os.path.join(image_dir, imageK), os.path.join(K_train_dir, imageK))
    shutil.copy(os.path.join(image_dir, imageD), os.path.join(D_train_dir, imageD))

#test verisi de eşit olsun
N = min(len(K_test), len(D_test))
K_test = K_test[:N]
D_test = D_test[:N]
print(f"Images in K(black) {len(K_test)} and D(blonde) {len(D_test)}")

K_test_dir = os.path.join(output_dir, "test/K")
D_test_dir = os.path.join(output_dir, "test_D")

os.makedirs(K_test_dir, exist_ok = True)
os.makedirs(D_test_dir, exist_ok = True)

for imageK, imageD in zip(K_test, D_test):
    shutil.copy(os.path.join(image_dir, imageK), os.path.join(K_test_dir, imageK))
    shutil.copy(os.path.join(image_dir, imageD), os.path.join(D_test_dir, imageD))

DATASET SINIFIMIZ

In [ ]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_ = None, unaligned = False, mode = "train"):
        #gelen transform fonksiyonu kullanılıyoe
        self.transform = transforms.Compose(transforms_)
        self.unaligned = unaligned

    self.files_K = sorted(glob.glob(os.path.join(root, "%s/K" %mode) + "/*.*"))
    self.files_D = sorted(glob.glob(os.path.join(root, "%s/D" %mode) + "/*.*"))

    def __getitem__(self,index):
        #eğitim yapılırken her veri alışında bu fonksiyona giriliyor ve veri ön işlemeden geçiriliyor
        item_K = self.transform(Image.open(self.files_K[index % len(self.files_K)]))

        if self.unaligned:
            # veriler eşli olmadığından dolayı rastgele bir foto seçiliyo
            item_D = self.transform(Image.open(self.files_D[random.randint(0, len(self.files_D) - 1)]))
        else:
            item_D = self.transform(Image.open(self.files_D[index % len(self.files_D)]))

        return {"K(black)": item_K, "D(blonde)": item_D}

    def __len__(self):
        return max(len(self.files_K), len(self.files_D))

GENERATOR BLOKLARI VE GENERATOR YAPAY SİNİR AĞI TASARIMI

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        conv_block = [
            
        ]